# PDF Ingestion Layer - Test Notebook

This notebook tests the PDF Ingestion Layer in isolation.

## Purpose
- Load a real PDF from the project
- Run the ingestion pipeline
- Inspect the output Document
- Validate metadata extraction
- Verify text quality

## What This Tests
✅ PDF loading and validation  
✅ Text extraction quality  
✅ Text normalization  
✅ Metadata extraction  
✅ LlamaIndex Document creation  

## What This Does NOT Test
❌ Chunking (that's Layer 2)  
❌ Embeddings (that's Layer 3)  
❌ Retrieval (that's Layer 3)  
❌ Agents (that's Layer 4)


---
## Setup


In [2]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent.parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")


Project root: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2


In [3]:
# Import the ingestion pipeline
from RAG.PDF_Ingestion.pdf_ingestion import create_ingestion_pipeline, PDFIngestionError

print("✅ PDF Ingestion module imported successfully")


✅ PDF Ingestion module imported successfully


---
## Test 1: Basic Ingestion


In [4]:
# Path to test PDF
pdf_path = project_root / "auto_claim_20_forms_FINAL.pdf"

print(f"Testing with PDF: {pdf_path}")
print(f"File exists: {pdf_path.exists()}")
print(f"File size: {pdf_path.stat().st_size / 1024:.1f} KB")


Testing with PDF: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2/auto_claim_20_forms_FINAL.pdf
File exists: True
File size: 44.9 KB


In [5]:
# Create ingestion pipeline
pipeline = create_ingestion_pipeline(document_type="insurance_claim_form")

print("✅ Pipeline created")


✅ Pipeline created


In [6]:
# Run ingestion
try:
    document = pipeline.ingest(str(pdf_path))
    print("✅ Ingestion successful!")
    print(f"Document type: {type(document)}")
    print(f"Document ID: {document.doc_id}")
except PDFIngestionError as e:
    print(f"❌ Ingestion failed: {e}")
    raise


✅ Ingestion successful!
Document type: <class 'llama_index.core.schema.Document'>
Document ID: 6e1c9a74673919ad


---
## Test 2: Inspect Metadata


In [7]:
# Display all metadata
import json

print("📋 DOCUMENT METADATA")
print("=" * 60)
print(json.dumps(document.metadata, indent=2))


📋 DOCUMENT METADATA
{
  "document_id": "6e1c9a74673919ad",
  "document_type": "insurance_claim_form",
  "source_file": "auto_claim_20_forms_FINAL.pdf",
  "source_path": "/Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2/auto_claim_20_forms_FINAL.pdf",
  "title": "auto_claim_20_forms_FINAL",
  "language": "en",
  "page_count": 40,
  "total_characters": 25417,
  "total_words": 3641,
  "total_paragraphs": 40,
  "avg_paragraph_length": 91.0,
  "has_headings": false,
  "dates_detected": [
    "2024-06-06",
    "2024-07-08",
    "2024-11-16",
    "2024-06-08",
    "2024-07-30",
    "2024-08-20",
    "2024-08-01",
    "2024-02-14",
    "2024-02-25",
    "2024-02-16"
  ],
  "times_detected": [
    "14:59",
    "15:15",
    "15:33",
    "09:29",
    "09:32",
    "09:31",
    "10:12",
    "14:54",
    "15:20",
    "10:59"
  ],
  "numeric_density": "medium",
  "ingested_at": "2025-12-13T10:56:37.198602Z",
  "ingestion_pipeline_version": "1.0"
}


In [8]:
# Validate required metadata fields
required_fields = [
    "document_id",
    "document_type",
    "source_file",
    "title",
    "language",
    "total_words",
    "total_characters",
    "total_paragraphs",
    "avg_paragraph_length",
    "has_headings",
    "dates_detected",
    "times_detected",
    "numeric_density",
    "ingested_at",
]

print("\n🔍 METADATA VALIDATION")
print("=" * 60)
missing_fields = []
for field in required_fields:
    if field in document.metadata:
        print(f"✅ {field}")
    else:
        print(f"❌ {field} - MISSING")
        missing_fields.append(field)

if missing_fields:
    print(f"\n⚠️  Missing {len(missing_fields)} required fields")
else:
    print("\n✅ All required metadata fields present")



🔍 METADATA VALIDATION
✅ document_id
✅ document_type
✅ source_file
✅ title
✅ language
✅ total_words
✅ total_characters
✅ total_paragraphs
✅ avg_paragraph_length
✅ has_headings
✅ dates_detected
✅ times_detected
✅ numeric_density
✅ ingested_at

✅ All required metadata fields present


---
## Test 3: Inspect Text Quality


In [9]:
# Display text statistics
print("📊 TEXT STATISTICS")
print("=" * 60)
print(f"Total characters: {len(document.text):,}")
print(f"Total words: {len(document.text.split()):,}")
print(f"Total lines: {len(document.text.split(chr(10))):,}")
print(f"Total paragraphs: {len([p for p in document.text.split(chr(10)*2) if p.strip()]):,}")


📊 TEXT STATISTICS
Total characters: 25,417
Total words: 3,641
Total lines: 79
Total paragraphs: 40


In [10]:
# Display first 500 characters
print("\n📄 FIRST 500 CHARACTERS OF CLEAN TEXT")
print("=" * 60)
print(document.text[:500])
print("\n[... text continues ...]")



📄 FIRST 500 CHARACTERS OF CLEAN TEXT
AUTO CLAIM FORM #1 BlueRiver Mutual SECTION 1 – CLAIMANT INFORMATION Name: Jon Mor Account Number: ACC9900057 Address: 100 Main Street, Sample City, ST 90000 Phone: (555) 100-2000 Email: jon.mor@example.com Date of Incident: 2024-06-06 Location: 10th Ave & 5th St, Sample City Injury: Yes (minor) Police Report: Yes SECTION 2 – CLAIM DETAILS Accident Type: Rear-end collision Severity: Minor Claim Status: Pending court Fraud Risk Score: 4 Internal Tag: TOW-FLAG-3 Assigned Adjuster: Linda Cooper SEC

[... text continues ...]


In [11]:
# Display last 500 characters
print("\n📄 LAST 500 CHARACTERS OF CLEAN TEXT")
print("=" * 60)
print("[... text continues ...]\n")
print(document.text[-500:])



📄 LAST 500 CHARACTERS OF CLEAN TEXT
[... text continues ...]

Witness Statement: Witness reported seeing the other car accelerate abruptly. Repair Estimate 1: $1260 Repair Estimate 2: $1645 Repair Shop Assigned: Horizon Collision Repair Repair Appointment Date: 2025-09-04

Hidden Note: Second witness: **Laura Vance** SECTION 5 – MINI TIMELINE OF EVENTS No timeline available for this claim. SECTION 6 – COURT DATE Court Date: N/A SECTION 7 – DECLARATION I declare that the information provided isaccurate. Signature: __________________________ Date: 2025-08-09


In [ ]:
# Show sample paragraphs
paragraphs = [p.strip() for p in document.text.split('\n\n') if p.strip()]

print(f"\n📄 SAMPLE PARAGRAPHS (showing 3 of {len(paragraphs)})")
print("=" * 60)

for i, para in enumerate(paragraphs[:3], 1):
    print(f"\nParagraph {i}:")
    print("-" * 60)
    # Show first 300 chars of paragraph
    if len(para) > 300:
        print(para[:300] + "...")
    else:
        print(para)


---
## Test 4: Text Quality Checks


In [12]:
# Check for common PDF extraction issues
print("🔍 TEXT QUALITY CHECKS")
print("=" * 60)

issues = []

# Check 1: Text is not empty
if not document.text.strip():
    issues.append("❌ Text is empty")
else:
    print("✅ Text is not empty")

# Check 2: Text has reasonable length
if len(document.text) < 100:
    issues.append("❌ Text is suspiciously short")
else:
    print("✅ Text has reasonable length")

# Check 3: No excessive whitespace
if '\n\n\n' in document.text:
    issues.append("⚠️  Text contains excessive newlines (3+)")
else:
    print("✅ No excessive newlines")

# Check 4: No form feed characters
if '\f' in document.text:
    issues.append("❌ Text contains form feed characters")
else:
    print("✅ No form feed characters")

# Check 5: Words are properly formed (no excessive single chars)
words = document.text.split()
single_char_words = sum(1 for w in words if len(w) == 1 and w.isalpha())
single_char_ratio = single_char_words / len(words) if words else 0
if single_char_ratio > 0.1:
    issues.append(f"⚠️  High ratio of single-character words ({single_char_ratio:.1%})")
else:
    print(f"✅ Low ratio of single-character words ({single_char_ratio:.1%})")

# Check 6: Has paragraph structure
if '\n\n' not in document.text:
    issues.append("⚠️  Text has no paragraph breaks")
else:
    print("✅ Text has paragraph structure")

# Summary
if issues:
    print(f"\n⚠️  Found {len(issues)} quality issues:")
    for issue in issues:
        print(f"  {issue}")
else:
    print("\n✅ All quality checks passed!")


🔍 TEXT QUALITY CHECKS
✅ Text is not empty
✅ Text has reasonable length
✅ No excessive newlines
✅ No form feed characters
✅ Low ratio of single-character words (0.6%)
✅ Text has paragraph structure

✅ All quality checks passed!


In [13]:
# Verify it's a proper LlamaIndex Document
from llama_index.core import Document as LlamaDocument

print("🔍 DOCUMENT OBJECT VALIDATION")
print("=" * 60)

# Check type
if isinstance(document, LlamaDocument):
    print("✅ Document is a LlamaIndex Document instance")
else:
    print(f"❌ Document is not a LlamaIndex Document (type: {type(document)})")

# Check required attributes
if hasattr(document, 'text'):
    print("✅ Document has 'text' attribute")
else:
    print("❌ Document missing 'text' attribute")

if hasattr(document, 'metadata'):
    print("✅ Document has 'metadata' attribute")
else:
    print("❌ Document missing 'metadata' attribute")

if hasattr(document, 'doc_id'):
    print("✅ Document has 'doc_id' attribute")
    print(f"   doc_id: {document.doc_id}")
else:
    print("❌ Document missing 'doc_id' attribute")

# Check doc_id matches metadata
if document.doc_id == document.metadata.get('document_id'):
    print("✅ doc_id matches metadata.document_id")
else:
    print("⚠️  doc_id does not match metadata.document_id")


🔍 DOCUMENT OBJECT VALIDATION
✅ Document is a LlamaIndex Document instance
✅ Document has 'text' attribute
✅ Document has 'metadata' attribute
✅ Document has 'doc_id' attribute
   doc_id: 6e1c9a74673919ad
✅ doc_id matches metadata.document_id


---
## Test 6: Error Handling


In [14]:
# Test with non-existent file
print("🔍 ERROR HANDLING TESTS")
print("=" * 60)

print("\nTest: Non-existent file")
try:
    pipeline.ingest("nonexistent.pdf")
    print("❌ Should have raised PDFIngestionError")
except PDFIngestionError as e:
    print(f"✅ Correctly raised PDFIngestionError: {e}")

print("\nTest: Non-PDF file")
try:
    pipeline.ingest("requirements.txt")
    print("❌ Should have raised PDFIngestionError")
except PDFIngestionError as e:
    print(f"✅ Correctly raised PDFIngestionError: {e}")


🔍 ERROR HANDLING TESTS

Test: Non-existent file
✅ Correctly raised PDFIngestionError: PDF file does not exist: nonexistent.pdf

Test: Non-PDF file
✅ Correctly raised PDFIngestionError: PDF file does not exist: requirements.txt


---
## Summary

This notebook has tested the PDF Ingestion Layer in isolation.

### What We Verified:
1. ✅ PDF file loading and validation
2. ✅ Text extraction from PDF
3. ✅ Text normalization and cleaning
4. ✅ Metadata extraction (document-level)
5. ✅ LlamaIndex Document object creation
6. ✅ Error handling for invalid inputs

### Next Layer:
**Layer 2: Chunking**
- Will take this Document as input
- Will create hierarchical nodes (Sections → Parent → Child)
- Will enrich metadata with chunk-level information
- Still NO embeddings or vector stores

### Notes:
- This layer is COMPLETE and ISOLATED
- It has NO dependencies on chunking, indexing, or agents
- It produces a clean Document ready for Layer 2
- All metadata is lightweight and document-level only
